# Clustering documents using topic modeling

The goal in clustering is to group together similar documents creating a number of different groups. To judge the similarity of the documents it is common for each document to be represented by a vector of weights which are assigned to each word in the document. These weights in most cases are the <a src=https://en.wikipedia.org/wiki/Tf%E2%80%93idf>tf-idf</a> frequencies of the words. Thus, an NxM dimensional matrix is created where N is the number of documents and M the dimensions of the vector space (number of words in the dictionary of the document collection). The end result of clustering is a number of clusters with each document assigned to a cluster.

Another method of organizing a document collection is topic modeling. In topic modeling each word of the dictionary is associated with a probability of occurence in a topic and each topic is associated with a probability of occurence in a document. Each document is represented by a vector of the probabilities of each topic in it. This means that topic modeling results to a NxM matrix representation of the collection where N is the number of documents and M is the number of topics. Since we have a vector space representation of the documents we can use it for clustering. 

# Topic modeling

We'll use <a src=http://radimrehurek.com/gensim/>Gensim</a> for topic modeling as it offers time and space efficient implementations of the topic modeling algorithms and it's free. It also supports parallelization although I don't make us of this feature here. With Gensim you can use either Latent Semantic Analysis (LSA) or Latent Dirichlet Allocation (LDA) to produce the topic vector space described above. We won't get into how these work now but you can find more information <a src=https://en.wikipedia.org/wiki/Latent_semantic_analysis>here</a> and <a src=https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation>here</a>. In this implementation i will use LSA as it is faster that LDA. It is also less accurate but the goal of this notebook is a proof of concept. 

The data we are going to use are greek WikiPedia articles xml formatted and bz2 compressed. You can download them from <a src=https://dumps.wikimedia.org/elwiki/20170601/elwiki-20170601-pages-meta-history.xml.bz2>here</a>. Gensim can work directly on the compressed collection.

First we need to create a Dictionary object, which is the dictionary of the document collection and associates each word to an id, and a MmCorpus object, which stores the corpus in a <a src=http://math.nist.gov/MatrixMarket/formats.html>Matrix Market</a> format. In a MmCorpus object each document can be represented by a vector of either integer count frequencies of it's word or tf-idf frequencies, but either way the resulting matrix is stored in the Matrix Market format.

I have saved the compressed WikiPedia XML file in a directory called data.

In [2]:
%run -m gensim.scripts.make_wikicorpus data/elwiki-20170720-pages-articles.xml.bz2 data/grwiki

2017-08-01 14:55:56,120 : INFO : running /home/theovasi/anaconda3/lib/python3.6/site-packages/gensim/scripts/make_wikicorpus.py data/elwiki-20170720-pages-articles.xml.bz2 data/grwiki
2017-08-01 14:55:56,323 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-08-01 14:56:25,444 : INFO : adding document #10000 to Dictionary(361468 unique tokens: ['αθλητισμός', 'είναι', 'συστηματική', 'σωματική', 'καλλιέργεια']...)
2017-08-01 14:56:46,820 : INFO : adding document #20000 to Dictionary(500714 unique tokens: ['αθλητισμός', 'είναι', 'συστηματική', 'σωματική', 'καλλιέργεια']...)
2017-08-01 14:57:05,578 : INFO : adding document #30000 to Dictionary(598622 unique tokens: ['αθλητισμός', 'είναι', 'συστηματική', 'σωματική', 'καλλιέργεια']...)
2017-08-01 14:57:24,443 : INFO : adding document #40000 to Dictionary(686151 unique tokens: ['αθλητισμός', 'είναι', 'συστηματική', 'σωματική', 'καλλιέργεια']...)
2017-08-01 14:57:42,056 : INFO : adding document #50000 to Dictionary(757946 uniq

The above script takes the compressed XML file as its first argument. The second argument is the prefix of the output files. When run this script creates the dictionary and saves it in /data/grwiki_wordids.txt.bz2 and the tf-idf representation of the corpus saved in Matrix Market format in /data/grwiki_tfidf.mm. Now lets load these files and create the needed objects.

First unzip the dictionary file:

In [5]:
!bzip2 -dk data/grwiki_wordids.txt.bz2

Create the Dictionary and MmCorpus objects by loading the files:

In [3]:
import logging, gensim, bz2
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO) # Allow gensim to print additional info.

dictionary = gensim.corpora.Dictionary.load_from_text('../data/grwiki_wordids.txt')
corpus = gensim.corpora.MmCorpus('../data/grwiki_tfidf.mm')

2017-08-03 11:41:02,035 : INFO : loaded corpus index from ../data/grwiki_tfidf.mm.index
2017-08-03 11:41:02,036 : INFO : initializing corpus reader from ../data/grwiki_tfidf.mm
2017-08-03 11:41:02,037 : INFO : accepted corpus with 122813 documents, 99121 features, 20926208 non-zero entries


Compute the LSA of the Greek WikiPedia:

In [4]:
lsi = gensim.models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary, num_topics=100)

2017-08-03 11:41:06,631 : INFO : using serial LSI version on this node
2017-08-03 11:41:06,633 : INFO : updating model with new documents
2017-08-03 11:41:30,159 : INFO : preparing a new chunk of documents
2017-08-03 11:41:31,077 : INFO : using 100 extra samples and 2 power iterations
2017-08-03 11:41:31,078 : INFO : 1st phase: constructing (99121, 200) action matrix
2017-08-03 11:41:32,597 : INFO : orthonormalizing (99121, 200) action matrix
2017-08-03 11:41:48,634 : INFO : 2nd phase: running dense svd on (200, 20000) matrix
2017-08-03 11:41:49,800 : INFO : computing the final decomposition
2017-08-03 11:41:49,801 : INFO : keeping 100 factors (discarding 22.315% of energy spectrum)
2017-08-03 11:41:50,003 : INFO : processed documents up to #20000
2017-08-03 11:41:50,031 : INFO : topic #0(14.900): 0.274*"έλληνας" + 0.217*"αμερικανός" + 0.199*"πολιτικός" + 0.188*"ηθοποιός" + 0.145*"συγγραφέας" + 0.134*"βασιλιάς" + 0.131*"γεννήσεις" + 0.129*"θάνατοι" + 0.128*"γρηγοριανό" + 0.127*"hμερολό

2017-08-03 11:43:59,315 : INFO : 2nd phase: running dense svd on (200, 20000) matrix
2017-08-03 11:44:00,288 : INFO : computing the final decomposition
2017-08-03 11:44:00,289 : INFO : keeping 100 factors (discarding 22.928% of energy spectrum)
2017-08-03 11:44:00,476 : INFO : merging projections: (99121, 100) + (99121, 100)
2017-08-03 11:44:02,576 : INFO : keeping 100 factors (discarding 12.315% of energy spectrum)
2017-08-03 11:44:02,999 : INFO : processed documents up to #100000
2017-08-03 11:44:03,005 : INFO : topic #0(31.376): 0.232*"φυσικά" + 0.217*"αστεροειδών" + 0.212*"jpl" + 0.209*"κύριας" + 0.209*"java" + 0.207*"αστεροειδής" + 0.207*"τροχιά" + 0.203*"ηλιακό" + 0.202*"απόλυτο" + 0.201*"ζώνης"
2017-08-03 11:44:03,009 : INFO : topic #1(26.170): 0.360*"ποδοσφαιριστές" + 0.138*"εθνική" + 0.135*"πρωτάθλημα" + 0.119*"px" + 0.106*"κύπελλο" + 0.102*"εθνικής" + 0.096*"ποδοσφαίρου" + 0.081*"αγώνες" + 0.080*"έλληνας" + 0.079*"ομάδες"
2017-08-03 11:44:03,012 : INFO : topic #2(22.064): -0.

Save LSI model for future use:

In [5]:
lsi.save('../data/model.lsi')

2017-08-02 14:42:32,157 : INFO : saving Projection object under ../data/model.lsi.projection, separately None
2017-08-02 14:42:32,940 : INFO : saved ../data/model.lsi.projection
2017-08-02 14:42:32,942 : INFO : saving LsiModel object under ../data/model.lsi, separately None
2017-08-02 14:42:32,943 : INFO : not storing attribute projection
2017-08-02 14:42:32,944 : INFO : not storing attribute dispatcher
2017-08-02 14:42:33,008 : INFO : saved ../data/model.lsi


We have created a model that can transform a vector from the tf-idf vector space to the topic vector space. This model extracted 400 topics from the documents. The first 10 topics are printed below. Each topic is represented by its 10 most contributing words (negative or positive).

In [5]:
lsi.print_topics(10)

2017-08-03 11:46:31,007 : INFO : topic #0(31.676): 0.230*"φυσικά" + 0.214*"αστεροειδών" + 0.209*"jpl" + 0.207*"κύριας" + 0.206*"java" + 0.205*"αστεροειδής" + 0.205*"τροχιά" + 0.201*"ηλιακό" + 0.200*"απόλυτο" + 0.199*"ζώνης"
2017-08-03 11:46:31,011 : INFO : topic #1(28.461): 0.305*"ποδοσφαιριστές" + 0.133*"πρωτάθλημα" + 0.129*"εθνική" + 0.124*"px" + 0.101*"κύπελλο" + 0.093*"εθνικής" + 0.086*"ποδοσφαίρου" + 0.083*"αγώνες" + 0.080*"έλληνας" + 0.079*"κόμμα"
2017-08-03 11:46:31,015 : INFO : topic #2(23.765): -0.594*"ποδοσφαιριστές" + -0.127*"πρωτάθλημα" + -0.123*"εθνική" + -0.114*"κύπελλο" + -0.100*"αγωνίστηκε" + -0.100*"ποδοσφαίρου" + -0.095*"γκολ" + -0.095*"εθνικής" + 0.087*"χωριό" + -0.086*"λιγκ"
2017-08-03 11:46:31,018 : INFO : topic #3(19.716): -0.228*"κόμμα" + 0.225*"χωριό" + 0.223*"δήμος" + -0.189*"εκλογές" + 0.186*"νομού" + 0.157*"δήμο" + 0.149*"κατοίκους" + 0.148*"δήμου" + 0.141*"απογραφή" + -0.133*"βουλευτές"
2017-08-03 11:46:31,022 : INFO : topic #4(19.473): 0.585*"px" + -0.458*"

[(0,
  '0.230*"φυσικά" + 0.214*"αστεροειδών" + 0.209*"jpl" + 0.207*"κύριας" + 0.206*"java" + 0.205*"αστεροειδής" + 0.205*"τροχιά" + 0.201*"ηλιακό" + 0.200*"απόλυτο" + 0.199*"ζώνης"'),
 (1,
  '0.305*"ποδοσφαιριστές" + 0.133*"πρωτάθλημα" + 0.129*"εθνική" + 0.124*"px" + 0.101*"κύπελλο" + 0.093*"εθνικής" + 0.086*"ποδοσφαίρου" + 0.083*"αγώνες" + 0.080*"έλληνας" + 0.079*"κόμμα"'),
 (2,
  '-0.594*"ποδοσφαιριστές" + -0.127*"πρωτάθλημα" + -0.123*"εθνική" + -0.114*"κύπελλο" + -0.100*"αγωνίστηκε" + -0.100*"ποδοσφαίρου" + -0.095*"γκολ" + -0.095*"εθνικής" + 0.087*"χωριό" + -0.086*"λιγκ"'),
 (3,
  '-0.228*"κόμμα" + 0.225*"χωριό" + 0.223*"δήμος" + -0.189*"εκλογές" + 0.186*"νομού" + 0.157*"δήμο" + 0.149*"κατοίκους" + 0.148*"δήμου" + 0.141*"απογραφή" + -0.133*"βουλευτές"'),
 (4,
  '0.585*"px" + -0.458*"ποδοσφαιριστές" + 0.169*"πρωτάθλημα" + 0.111*"κύπελλο" + 0.107*"ομάδες" + 0.103*"αγώνες" + 0.103*"ολυμπιακός" + 0.093*"παναθηναϊκός" + 0.090*"ανδρών" + -0.083*"χωριό"'),
 (5,
  '-0.370*"ταινίες" + 0.294*

Apply the LSI tranformation to the whole collection that is represented in
the corpus object in tf-df form.

In [6]:
corpus_lsi = lsi[corpus]

We have managed to create a topic space representation of the Greek 
WikiPedia and reduced teh dimensions of the vector space from NxM (N is
the number of documents and M is sthe number of words in the dictionary) 
to Nx100.

A document is represented in the new vector sapce like this:

In [7]:
corpus_lsi[0]

[(0, 0.043097616110488325),
 (1, 0.10538208571697072),
 (2, 0.049693755551678806),
 (3, 0.013177885592721289),
 (4, 0.022210603056187848),
 (5, -0.047895680123100901),
 (6, 0.02779818360480461),
 (7, -0.055389267100266051),
 (8, -0.028944547760499775),
 (9, 0.059686660962608683),
 (10, -0.054395228223970249),
 (11, -0.043106698028264161),
 (12, 0.046429293537655916),
 (13, 0.028013281470275854),
 (14, 0.035825578529825368),
 (15, 0.026825692676210662),
 (16, 0.011081979206418789),
 (17, 0.011744732711968509),
 (18, -0.051268652323500509),
 (19, 0.043715181426642165),
 (20, -0.0096210998635424903),
 (21, -0.014752740761021722),
 (22, -0.010672577099673727),
 (23, 0.043502740153385573),
 (24, -0.023750172517982331),
 (25, 0.014728183535870146),
 (26, 0.02601227250022865),
 (27, 0.027615780775637599),
 (28, 0.030004526702247201),
 (29, -0.0086155325075867786),
 (30, 0.016341604498913483),
 (31, -0.045182979662826972),
 (32, 0.025455021008494342),
 (33, -0.025658191464714552),
 (34, 0.0035

The next step is the clustering.

# Clustering

We are going to use scikit-learn for the clustering. Scikit's algorithms require for the vector space matrix to be in a sparse matrix format. Gensim 
provides a function to do just that.

In [8]:
lsi_sparse = gensim.matutils.corpus2csc(corpus_lsi)
lsi_sparse.shape

(100, 122813)

This sparse matrix has the documents as the columns. We need to reverse it
before fitting the model, because sklearn expects the documents to be in rows.

In [9]:
import scipy
lsi_sparse = scipy.sparse.csc_matrix.transpose(lsi_sparse)
lsi_sparse.shape

(122813, 100)

I am using <a src=https://en.wikipedia.org/wiki/K-means_clustering>k-means</a> to create 8 clusters without any refinements apart from a maximum iteration number. MiniBatchKMeans is a more computationally effiecient implementation of k-means, that uses a fixed size subsample of the collection in each iteration.

In [10]:
from sklearn.cluster import MiniBatchKMeans
kmodel = MiniBatchKMeans(n_clusters=8, max_iter=10, verbose=True)

In [ ]:
Finally fit the k-means model on the documents x topic weights matrix we
created above. 

In [11]:
kmodel.fit(lsi_sparse) 

Init 1/3 with method: k-means++
Inertia for init 1/3: 25.764056
Init 2/3 with method: k-means++
Inertia for init 2/3: 25.381525
Init 3/3 with method: k-means++
Inertia for init 3/3: 23.299360
Minibatch iteration 1/12290: mean batch inertia: 0.094869, ewa inertia: 0.094869 
Minibatch iteration 2/12290: mean batch inertia: 0.079461, ewa inertia: 0.094844 
Minibatch iteration 3/12290: mean batch inertia: 0.089083, ewa inertia: 0.094835 
Minibatch iteration 4/12290: mean batch inertia: 0.081780, ewa inertia: 0.094814 
Minibatch iteration 5/12290: mean batch inertia: 0.089899, ewa inertia: 0.094806 
Minibatch iteration 6/12290: mean batch inertia: 0.091719, ewa inertia: 0.094801 
Minibatch iteration 7/12290: mean batch inertia: 0.103704, ewa inertia: 0.094815 
Minibatch iteration 8/12290: mean batch inertia: 0.066246, ewa inertia: 0.094769 
Minibatch iteration 9/12290: mean batch inertia: 0.078329, ewa inertia: 0.094742 
Minibatch iteration 10/12290: mean batch inertia: 0.101849, ewa inerti

Minibatch iteration 150/12290: mean batch inertia: 0.074904, ewa inertia: 0.092115 
Minibatch iteration 151/12290: mean batch inertia: 0.067760, ewa inertia: 0.092076 
Minibatch iteration 152/12290: mean batch inertia: 0.066583, ewa inertia: 0.092034 
Minibatch iteration 153/12290: mean batch inertia: 0.072471, ewa inertia: 0.092002 
Minibatch iteration 154/12290: mean batch inertia: 0.065788, ewa inertia: 0.091960 
Minibatch iteration 155/12290: mean batch inertia: 0.094243, ewa inertia: 0.091963 
Minibatch iteration 156/12290: mean batch inertia: 0.064424, ewa inertia: 0.091919 
Minibatch iteration 157/12290: mean batch inertia: 0.067222, ewa inertia: 0.091878 
Minibatch iteration 158/12290: mean batch inertia: 0.085169, ewa inertia: 0.091867 
Minibatch iteration 159/12290: mean batch inertia: 0.100383, ewa inertia: 0.091881 
Minibatch iteration 160/12290: mean batch inertia: 0.085629, ewa inertia: 0.091871 
Minibatch iteration 161/12290: mean batch inertia: 0.079591, ewa inertia: 0.

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=10, max_no_improvement=10, n_clusters=8,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=True)

We have successfully trained the k-means model.

# Visualization

Using the k-means model we just trained we can get the topic space representation of the centers of the clusters.

In [12]:
centers = kmodel.cluster_centers_.argsort()[:, ::-1]

We have each cluster center vector in ascending topic weight.

In [13]:
print(centers[0])

[ 1  4  6  7  3  0 18 19  5 27 15 17 11 59 41 57 16 80 48 66 25 62 94 77 34
 46 49 29 70 90 82 92 36 42 60 98 30 93 81 51 23 85 56 44 88 35 71 83 14 47
 50 67 97 74 79 40 38 24 65 55 73 54 28 95 84 78 99 53 10 63 96 91 45 26 75
 64 89 33 39 43 72 76 12 87 61 52 58 37 32 86 68 31 69 20  9 21 22 13  8  2]


The vector printed above is the topic space representation of the first cluster. We can see that topic number 7 is the topic with the highest weight for this cluster. One way to visualize the clustering is to do the above for all the clusters, pick the 3 most contributing topics and display
some information about them. Gensim provides a way to get information about a topic by printing the most contributing words for the direction of the topic. This contribution can be positive or negative.

In [20]:
for i, center in enumerate(centers):
    # Pick the 3 most contributing topics.
    print("Cluster: " + str(i))
    for j in range(3):
        # Print top 10 most contributing words.
        print("    Topic " + str(j) + ": " +
              lsi.print_topic(center[j], topn=5))

Cluster: 0
    Topic 0: 0.305*"ποδοσφαιριστές" + 0.133*"πρωτάθλημα" + 0.129*"εθνική" + 0.124*"px" + 0.101*"κύπελλο"
    Topic 1: 0.585*"px" + -0.458*"ποδοσφαιριστές" + 0.169*"πρωτάθλημα" + 0.111*"κύπελλο" + 0.107*"ομάδες"
    Topic 2: -0.712*"px" + -0.296*"ποδοσφαιριστές" + 0.219*"ολυμπιονίκες" + 0.174*"αρχαίοι" + 0.163*"πρωτάθλημα"
Cluster: 1
    Topic 0: -0.228*"κόμμα" + 0.225*"χωριό" + 0.223*"δήμος" + -0.189*"εκλογές" + 0.186*"νομού"
    Topic 1: 0.305*"ποδοσφαιριστές" + 0.133*"πρωτάθλημα" + 0.129*"εθνική" + 0.124*"px" + 0.101*"κύπελλο"
    Topic 2: -0.594*"ποδοσφαιριστές" + -0.127*"πρωτάθλημα" + -0.123*"εθνική" + -0.114*"κύπελλο" + -0.100*"αγωνίστηκε"
Cluster: 2
    Topic 0: 0.305*"ποδοσφαιριστές" + 0.133*"πρωτάθλημα" + 0.129*"εθνική" + 0.124*"px" + 0.101*"κύπελλο"
    Topic 1: 0.230*"φυσικά" + 0.214*"αστεροειδών" + 0.209*"jpl" + 0.207*"κύριας" + 0.206*"java"
    Topic 2: -0.594*"ποδοσφαιριστές" + -0.127*"πρωτάθλημα" + -0.123*"εθνική" + -0.114*"κύπελλο" + -0.100*"αγωνίστηκε"
Cluste